In [1]:
import string
from pathlib import Path
from collections import defaultdict
import json
import numpy as np
import math
file = open('./input.txt', 'r')

sensors_reading = []
all_x = []
all_y = []

for line in file.readlines():
    line = line.replace('\n','')
    # Sensor at x=98246, y=1908027: closest beacon is at x=1076513, y=2000000
    substrings = line.split('=')[1:]
    sens_x = int(substrings[0].split(',')[0])
    sens_y = int(substrings[1].split(':')[0])
    beacon_x = int(substrings[2].split(',')[0])
    beacon_y = int(substrings[3])    

    sensors_reading.append([(sens_x, sens_y), (beacon_x, beacon_y)])
    all_x.extend([sens_x, beacon_x])
    all_y.extend([sens_y, beacon_y])

print(f'num readings = {len(sensors_reading)}')

min_x = min(all_x)
max_x = max(all_x)
min_y = min(all_y)
max_y = max(all_y)
print(f'grid x is {min_x}-{max_x}')
print(f'grid y is {min_y}-{max_y}')


num readings = 24
grid x is 13166-4044141
grid y is -494417-4136840


In [2]:
def manh_dist(p1,p2):
    return abs(p1[0]-p2[0])+abs(p1[1]-p2[1])

In [3]:
safe_pos = set()
for i, reading in enumerate(sensors_reading):

    sens_beac_dist = manh_dist( reading[0],reading[1] )
    
    min_x = reading[0][0]-sens_beac_dist   
    max_x = reading[0][0]+sens_beac_dist    
    min_y = reading[0][1]-sens_beac_dist    
    max_y = reading[0][1]+sens_beac_dist

    # min_x = max(min_x, 0)
    # min_y = max(min_y, 0)

    print(f'for reading num {i}, checking pos with x in ({min_x}-{max_x}), y in ({min_y}-{max_y})')

    if 2000000<min_y or 2000000>max_y:
        continue
    else:
        y = 2000000
        for x in range(min_x,max_x+1):
            if manh_dist( reading[0], (x,y)) <= sens_beac_dist:
                safe_pos.add((x,y))
    # for x in range(min_x,max_x+1):
    #     for y in range(min_y,max_y+1):
    #         if manh_dist( reading[0], (x,y)) < sens_beac_dist:
    #             safe_pos.append((x,y))

print(len(safe_pos))

for reading in sensors_reading:
    if reading[1] in safe_pos:
        safe_pos.discard(reading[1])

print(len(safe_pos))

for reading num 0, checking pos with x in (-971994-1168486), y in (837787-2978267)
for reading num 1, checking pos with x in (992660-1686078), y in (1737144-2430562)
for reading num 2, checking pos with x in (-725464-2083818), y in (1602664-4411946)
for reading num 3, checking pos with x in (-145377-185901), y in (3812658-4143936)
for reading num 4, checking pos with x in (2455040-4065290), y in (1463830-3074080)
for reading num 5, checking pos with x in (1906683-3248667), y in (2391592-3733576)
for reading num 6, checking pos with x in (2883452-4483174), y in (1929276-3528998)
for reading num 7, checking pos with x in (-593048-2705872), y in (-1278819-2020101)
for reading num 8, checking pos with x in (2731090-2923470), y in (1730905-1923285)
for reading num 9, checking pos with x in (13477-3267439), y in (2327543-5581505)
for reading num 10, checking pos with x in (883772-3395996), y in (-93923-2418301)
for reading num 11, checking pos with x in (2913371-4641529), y in (2850425-45785

In [4]:
from tqdm import tqdm
#possible_pos = np.ones((4000001,4000001), dtype=bool)
safe_x = set()
safe_y = set()

sens_dist = []

for i, reading in enumerate(sensors_reading):

    sens_beac_dist = manh_dist( reading[0],reading[1] )

    sens_dist.append((reading[0],sens_beac_dist))
    print(f'for reading num {i}, distance from {reading[0]} is {sens_beac_dist}')

    # min_x = reading[0][0]-sens_beac_dist   
    # max_x = reading[0][0]+sens_beac_dist    
    # min_y = reading[0][1]-sens_beac_dist    
    # max_y = reading[0][1]+sens_beac_dist

    # min_x = max(min_x, 0)
    # min_y = max(min_y, 0)
    # max_x = max(max_x, 4000000)
    # max_y = max(max_y, 4000000)

    # print(f'checking pos with x in ({min_x}-{max_x}), y in ({min_y}-{max_y})')

for y in tqdm(range(0,4000001)):
    ranges = []
    for sens, dist in sens_dist:
        range_dim = dist-abs(sens[1]-y)
        if range_dim>0:
            range_b = (max(0,sens[0]-range_dim), min(4000000,sens[0]+range_dim))
            ranges.append(range_b)

    glob_range = []
    for begin,end in sorted(ranges):
        if glob_range and glob_range[-1][1] >= begin - 1:
            glob_range[-1][1] = max(glob_range[-1][1],end)
        else:
            glob_range.append([begin, end])

    if len(glob_range)>1:
        print(glob_range)
        assert len(glob_range)==2
        right_x = glob_range[0][1]+1
        right_y = y
        break
    elif glob_range[0][0]>0:
        print(glob_range)
        assert glob_range[0][0]==1
        right_x = 0
        right_y = y
    elif glob_range[0][1]<4000000:
        print(glob_range)
        assert glob_range[0][1]==3999999
        right_x = 4000000
        right_y = y
        break

print(f'the only possible position is ({right_x},{right_y})')
print(f'tuning frequency is {right_x*4000000 + right_y}')


for reading num 0, distance from (98246, 1908027) is 1070240
for reading num 1, distance from (1339369, 2083853) is 346709
for reading num 2, distance from (679177, 3007305) is 1404641
for reading num 3, distance from (20262, 3978297) is 165639
for reading num 4, distance from (3260165, 2268955) is 805125
for reading num 5, distance from (2577675, 3062584) is 670992
for reading num 6, distance from (3683313, 2729137) is 799861
for reading num 7, distance from (1056412, 370641) is 1649460
for reading num 8, distance from (2827280, 1827095) is 96190
for reading num 9, distance from (1640458, 3954524) is 1626981
for reading num 10, distance from (2139884, 1162189) is 1256112
for reading num 11, distance from (3777450, 3714504) is 864079
for reading num 12, distance from (1108884, 2426713) is 459084
for reading num 13, distance from (2364307, 20668) is 1123051
for reading num 14, distance from (3226902, 2838842) is 562131
for reading num 15, distance from (22804, 3803886) is 342592
for rea

 66%|██████▋   | 2658764/4000001 [00:22<00:11, 120547.23it/s]

[[0, 2844847], [2844849, 4000000]]
the only possible position is (2844848,2658764)
tuning frequency is 11379394658764
